<a href="https://colab.research.google.com/github/joshuacalloway/dsc540groupproject/blob/main/StartingTrumpTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request, json 

In [2]:
with urllib.request.urlopen("https://www.thetrumparchive.com/latest-tweets") as url:
    data = json.loads(url.read().decode())
    print(data)

[{'id': '1322297825334497280', 'text': 'Joe Biden will lock down our entire Country. Biden will make you a prisoner in your own home, and a prisoner in your own Country. Biden’s plan will kill the American Dream—Get your friends, get your family, get your neighbors and GET OUT AND VOTE! #MAGA https://t.co/fH8Iq6i8d3', 'isRetweet': False, 'isDeleted': False, 'device': 'Twitter for iPhone', 'favorites': '27981', 'retweets': '8693', 'date': '2020-10-30T22:02:23.000Z'}, {'id': '1322295405313323009', 'text': 'I’ve done more in 47 months than Joe Biden has done in 47 years! A vote for Republicans is a vote for the American Dream, and the party of Abraham Lincoln! #MAGA https://t.co/gsFSgh2KPc', 'isRetweet': False, 'isDeleted': False, 'device': 'Twitter for iPhone', 'favorites': '31587', 'retweets': '6584', 'date': '2020-10-30T21:52:46.000Z'}, {'id': '1322294301183782912', 'text': 'The Election should end on November 3rd., not weeks later!', 'isRetweet': False, 'isDeleted': False, 'device': '

In [9]:
data[0]['text']

'Joe Biden will lock down our entire Country. Biden will make you a prisoner in your own home, and a prisoner in your own Country. Biden’s plan will kill the American Dream—Get your friends, get your family, get your neighbors and GET OUT AND VOTE! #MAGA https://t.co/fH8Iq6i8d3'

In [15]:
get_tweet_text = lambda tweet : tweet['text']

tweets = map(get_tweet_text, data)
print(list(tweets))

['Joe Biden will lock down our entire Country. Biden will make you a prisoner in your own home, and a prisoner in your own Country. Biden’s plan will kill the American Dream—Get your friends, get your family, get your neighbors and GET OUT AND VOTE! #MAGA https://t.co/fH8Iq6i8d3', 'I’ve done more in 47 months than Joe Biden has done in 47 years! A vote for Republicans is a vote for the American Dream, and the party of Abraham Lincoln! #MAGA https://t.co/gsFSgh2KPc', 'The Election should end on November 3rd., not weeks later!', 'This election is a choice between a Trump Super Boom or a Biden Depression, and it’s between a safe vaccine or a devastating Biden lockdown! https://t.co/FkOHz0XUS2 https://t.co/IEPULUXAXe', 'One of the most important issues is LAW &amp; ORDER. Biden stands with the rioters &amp; looters - I stand with the HEROES of law enforcement. If Biden wins, the rioters in the streets will be running your federal government. No city, no family, and no community will be saf

https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/analyze-tweet-sentiment-in-python/

In [12]:
import tweepy as tw
from textblob import TextBlob

In [13]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
tweets